# Universo / Target

## Definición de target

Target = 1 cuando el alumno no curso ni rindió finales por 2 ciclos lectivos consecutivos

## Dataset a armar

ejemplo

| Alumno | Ciclo | Target |
| --- | --- | --- |
| 1 | 2010 | 0 |
| 1 | 2011 | 0 |
| 1 | 2012 | 1 |
| 1 | 2013 | 1 |
| 2 | 2012 | 0 |
| 2 | 2013 | 0 |
| 2 | 2014 | 0 |
| 2 | 2015 | 0 |

- ciclo minimo es el año de ingreso del alumno
- ciclo maximo es el año de egreso del alumno o del ultimo registro (cursadas o finales) + 2




In [2]:
import pandas as pd
import numpy as np

In [3]:
from ydata_profiling import ProfileReport

/Users/sjaremczuk/Library/Caches/pypoetry/virtualenvs/tesis-ds-polkNLC9-py3.9/lib/python3.9/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
/Users/sjaremczuk/Library/Caches/pypoetry/virtualenvs/tesis-ds-polkNLC9-py3.9/lib/python3.9/site-packages/visions/backends/shared/nan_handling.py:51: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def hasna(x: np.ndarray) -> bool:


## Datos de entrada

In [28]:
curdf = pd.read_pickle('../data/raw_cleaned/curdf_01.pkl')
print('cur shape: ',curdf.shape)
curdf_comp = pd.read_pickle('../data/raw_cleaned/curdf_complementario_01.pkl')
print('cur_comp shape: ',curdf_comp.shape)
datdf = pd.read_pickle('../data/raw_cleaned/datdf_01.pkl')
print('datdf shape: ',datdf.shape)
findf = pd.read_pickle('../data/raw_cleaned/findf_01.pkl')
print('fin shape: ',findf.shape)

cur shape:  (510290, 8)
cur_comp shape:  (79565, 8)
datdf shape:  (22157, 5)
fin shape:  (475362, 7)


## Armar Dataset con datos de cursadas (base + complementario) y finales

In [5]:
cur = pd.concat([curdf,curdf_comp],axis=0)
cur.head()

,codigo_alumno,curso,codigo_materia,departamento,turno,ciclo_lectivo,cant_cursada,cant_total_cursadas
4494,10035,K4153,950310,Unidad Docente Básica Legislación y Economía,Noche,2006.0,1.0,1
4497,10035,K3051,952028,NaN,Noche,2007.0,1.0,1
4492,10035,K3001,952030,NaN,Mañana,2006.0,1.0,1
4498,10035,K3071,952031,NaN,Noche,2007.0,1.0,1
4502,10035,K4051,952033,Departamento de Ing. en Sistemas de Información,Noche,2008.0,1.0,1


In [6]:
cur.shape

(589855, 8)

In [7]:
cur.tail()

,codigo_alumno,curso,codigo_materia,departamento,turno,ciclo_lectivo,cant_cursada,cant_total_cursadas
18038,9531,S/D,82060,S/D,S/D,2010.0,1.0,1
18039,9531,S/D,82062,S/D,S/D,2010.0,1.0,1
18085,9535,S/D,82036,S/D,S/D,2011.0,1.0,1
18839,9774,S/D,82025,S/D,S/D,2009.0,1.0,1
19551,9931,S/D,952072,S/D,S/D,2006.0,1.0,1


In [8]:
findf.head()

,codigo_alumno,codigo_materia,anio_academico,nota_num,nota_cual,nota_txt,cant_finales
20181,10035,950309,2006.0,2.0,NaN,No Aprobada,1.0
20182,10035,950309,2006.0,4.0,NaN,Aprobada,2.0
20183,10035,950310,2006.0,7.0,NaN,Aprobada,1.0
20173,10035,950611,2004.0,4.0,NaN,Aprobada,1.0
20166,10035,950701,2003.0,4.0,NaN,Aprobada,1.0


In [11]:
df = pd.concat( 
    [cur[['codigo_alumno', 'ciclo_lectivo']].copy().drop_duplicates(), 
    findf[['codigo_alumno', 'anio_academico']].copy().drop_duplicates().rename(columns={'anio_academico':'ciclo_lectivo'})], axis=0). \
    drop_duplicates().sort_values(['codigo_alumno', 'ciclo_lectivo']).reset_index(drop=True)


In [12]:
df.head()

,codigo_alumno,ciclo_lectivo
0,10035,2002.0
1,10035,2003.0
2,10035,2004.0
3,10035,2005.0
4,10035,2006.0


In [13]:
df.shape

(147536, 2)

In [14]:
df.duplicated().sum()

0

In [16]:
# traigo el año de ingreso para cada alumno

datdf.head()

,codigo_alumno,sexo,anio_nacimiento,anio_ingreso,codigo_carrera_ingreso
0,27,M,1974,1993,K
1,53,M,1974,1999,K
2,58,M,1974,1995,K
3,72,M,1974,1995,K
4,86,M,1974,1999,K


In [17]:
aux = datdf[['codigo_alumno', 'anio_ingreso']].drop_duplicates().copy()

df = pd.merge(df, aux, on='codigo_alumno', how='left')

df.head()

,codigo_alumno,ciclo_lectivo,anio_ingreso
0,10035,2002.0,2002
1,10035,2003.0,2002
2,10035,2004.0,2002
3,10035,2005.0,2002
4,10035,2006.0,2002


In [18]:
df.shape

(147536, 3)

In [21]:
# calcular el tiempo que pasa entre un ciclo lectivo y el siguiente para un mismo alumno
aux = df.copy()
aux['delta_ciclo_lectivo'] = aux.groupby('codigo_alumno')['ciclo_lectivo'].transform(lambda x: x - x.shift(1))
aux.head(20)

,codigo_alumno,ciclo_lectivo,anio_ingreso,delta_ciclo_lectivo
0,10035,2002.0,2002,NaN
1,10035,2003.0,2002,1.0
2,10035,2004.0,2002,1.0
3,10035,2005.0,2002,1.0
4,10035,2006.0,2002,1.0
5,10035,2007.0,2002,1.0
6,10035,2008.0,2002,1.0
7,10035,2009.0,2002,1.0
8,10036,2003.0,2003,NaN
9,10036,2004.0,2003,1.0


In [24]:
(aux.delta_ciclo_lectivo > 2).value_counts()

False    144245
True       3291
Name: delta_ciclo_lectivo, dtype: int64

In [25]:
3291/144245

0.02281534888557662

### conclusión

tenemos un 2,28% de desertores

dataset desbalanceado


## Armar Dataset con datos de cursadas (base) y finales

In [41]:
df = pd.concat( 
    [curdf[['codigo_alumno', 'ciclo_lectivo']].copy().drop_duplicates(), 
    findf[['codigo_alumno', 'anio_academico']].copy().drop_duplicates().rename(columns={'anio_academico':'ciclo_lectivo'})], axis=0). \
    drop_duplicates().sort_values(['codigo_alumno', 'ciclo_lectivo']).reset_index(drop=True)


In [42]:
df.shape

(147536, 2)

In [43]:
# calcular el tiempo que pasa entre un ciclo lectivo y el siguiente para un mismo alumno
aux = df.copy()
aux['delta_ciclo_lectivo'] = aux.groupby('codigo_alumno')['ciclo_lectivo'].transform(lambda x: x - x.shift(1))
aux.head(20)

,codigo_alumno,ciclo_lectivo,delta_ciclo_lectivo
0,10035,2002.0,NaN
1,10035,2003.0,1.0
2,10035,2004.0,1.0
3,10035,2005.0,1.0
4,10035,2006.0,1.0
5,10035,2007.0,1.0
6,10035,2008.0,1.0
7,10035,2009.0,1.0
8,10036,2003.0,NaN
9,10036,2004.0,1.0


In [48]:
(aux.delta_ciclo_lectivo > 2).value_counts()

False    144245
True       3291
Name: delta_ciclo_lectivo, dtype: int64

### conclusion

da la misma proporcion usando o no las cursadas complementarias

## Análsis de dataset si se hace a nivel alumno

### Contemplando solo el ultimo ciclo

In [50]:
# agrupar aux por alumno y filtrar fila con maximo ciclo lectivo
aux_alumno = aux.sort_values(['codigo_alumno', 'ciclo_lectivo']).groupby('codigo_alumno').last().reset_index().copy()

In [51]:
aux_alumno.head()

,codigo_alumno,ciclo_lectivo,delta_ciclo_lectivo
0,10035,2009.0,1.0
1,10036,2007.0,3.0
2,10039,2007.0,1.0
3,10042,2008.0,1.0
4,10044,2007.0,1.0


In [52]:

(aux_alumno.delta_ciclo_lectivo > 2).value_counts()

False    20919
True      1238
Name: delta_ciclo_lectivo, dtype: int64

In [53]:
1238/20919

0.05918064917061045

#### conclusión

si se hace a nivel alumno tenemos un desbalance del 6%


### Contemplando si alguna vez desertó a pensar de que despues retomó

In [54]:
# agrupar aux por alumno y quedarme con el máximo valor de delta_ciclo_lectivo
aux_alumno = aux.sort_values(['codigo_alumno', 'ciclo_lectivo']).groupby('codigo_alumno').max().reset_index().copy()

In [55]:
aux_alumno.head()

,codigo_alumno,ciclo_lectivo,delta_ciclo_lectivo
0,10035,2009.0,1.0
1,10036,2007.0,3.0
2,10039,2007.0,1.0
3,10042,2008.0,1.0
4,10044,2007.0,1.0


In [56]:

(aux_alumno.delta_ciclo_lectivo > 2).value_counts()

False    19388
True      2769
Name: delta_ciclo_lectivo, dtype: int64

In [57]:
2769/19388

0.14282030121724779

#### conclusión

- si se considera si desertó al menos una vez a pesar de que dsps retomó el desbalance es del 14,3%

# Nexts steps

1. por ahora no se contempla si un alumno regular abandona repentinamente la carrera xq al no haber registro siguiente, no se sabe si es un abandono y no se registra

2. en la condicion 1, hay q tener en cuenta que el alumno no haya terminado la carrera porque sería un falso desertor (en algunas carreras es obligatorio terminar con la materia proyecto final y en otras no. averiguar)

# Analisis de distribución de carreras

In [58]:
curdf.columns

Index(['codigo_alumno', 'curso', 'codigo_materia', 'departamento', 'turno',
       'ciclo_lectivo', 'cant_cursada', 'cant_total_cursadas'],
      dtype='object')

In [59]:
datdf.columns

Index(['codigo_alumno', 'sexo', 'anio_nacimiento', 'anio_ingreso',
       'codigo_carrera_ingreso'],
      dtype='object')

In [60]:
findf.columns

Index(['codigo_alumno', 'codigo_materia', 'anio_academico', 'nota_num',
       'nota_cual', 'nota_txt', 'cant_finales'],
      dtype='object')

In [61]:
datdf.codigo_carrera_ingreso.value_counts()

K     8702
R     3278
S     2822
V     2475
O     2045
I     1197
Q      928
TB     270
U      257
W      177
TU       5
B        1
Name: codigo_carrera_ingreso, dtype: int64

In [64]:
curdf.curso.apply(lambda x: x[0]).value_counts()

K    135468
Z    128557
V     49904
S     49694
R     43644
O     41634
I     29608
Q     15988
X      5595
M      2799
A      2766
W      2702
U       691
1       542
k       425
4        84
2        82
T        58
5        20
B        16
C         4
E         4
P         2
D         1
L         1
F         1
Name: curso, dtype: int64

In [65]:
curdf.columns

Index(['codigo_alumno', 'curso', 'codigo_materia', 'departamento', 'turno',
       'ciclo_lectivo', 'cant_cursada', 'cant_total_cursadas'],
      dtype='object')

In [67]:
curdf[~curdf.departamento.duplicated()][['curso','departamento']]

,curso,departamento
4494,K4153,Unidad Docente Básica Legislación y Economía
4497,K3051,NaN
4502,K4051,Departamento de Ing. en Sistemas de Información
4511,K1071,Unidad Docente Básica Química
4523,Z2055,Unidad Docente Básica Matemática
4609,R1031,"Departamento de Ing. Mecánica, Departamento de..."
4610,R1042,Unidad Docente Básica Cultura e Idiomas
4667,K2031,Unidad Docente Básica Física
124,I5054,Departamento de Ing. Industrial
5827,R6051,Departamento de Ing. Electrónica
